# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

> You do not need to run the following cells if you are running this notebook locally. 

In [1]:
#!pip install -qU langchain langchain-openai langchain-cohere rank_bm25

We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [2]:
#!pip install -qU qdrant-client

We'll also provide our OpenAI key, as well as our Cohere API key.

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [4]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [5]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

--2025-05-16 15:19:37--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connexion à raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 19628 (19K) [text/plain]
Sauvegarde en : « john_wick_1.csv »

john_wick_1.csv     100%[===================>]  19,17K  --.-KB/s    ds 0,006s  

2025-05-16 15:19:37 (3,21 MB/s) — « john_wick_1.csv » sauvegardé [19628/19628]

--2025-05-16 15:19:37--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connexion à raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443… connecté.
requête HTTP transmise, en atte

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [6]:
import pandas as pd

df = pd.read_csv("john_wick_4.csv", index_col= 0)
df


,Review_Date,Author,Rating,Review_Title,Review,Review_Url
0,23 May 2023,siderite,4.0,What a pointless film\n,Imagine a video game where you are shooting ba...,/review/rw9073117/?ref_=tt_urv
1,30 March 2023,neil-476,5.0,There is such a thing as too much\n,"The Table, the international crminal brotherho...",/review/rw8960544/?ref_=tt_urv
2,25 March 2023,BA_Harrison,4.0,It got on my wick.\n,The first three John Wick films came in fairly...,/review/rw8950606/?ref_=tt_urv
3,23 May 2023,namob-43673,3.0,I was rolling my eyes the whole time... all 3...,These John Wick movies can be sort of fun in t...,/review/rw9072963/?ref_=tt_urv
4,24 March 2023,fciocca,4.0,John Wick became the parody of himself. The t...,I went to the cinema with great expectations. ...,/review/rw8948738/?ref_=tt_urv
5,2 April 2023,skyhawk747,4.0,Am I missing something here?\n,What is all the raving about with this movie? ...,/review/rw8967740/?ref_=tt_urv
6,24 March 2023,IMDbKeepsDeletingMyReviews,5.0,"""Yeah.""\n",In this fourth installment of 8711's successfu...,/review/rw8947952/?ref_=tt_urv
7,23 April 2023,antti-eskelinen-329-929792,4.0,I don't understand the great scores of this m...,In my opinion this is by far the worst movie o...,/review/rw9011753/?ref_=tt_urv
8,28 May 2023,dstan-71445,2.0,Disappointed.\n,"Very much over rated. Repetitive, tiring and i...",/review/rw9082993/?ref_=tt_urv
9,30 March 2023,drjgardner,2.0,"Ridiculous, boring and pathetic...\n",...all at the same time. This hybrid comic boo...,/review/rw8959398/?ref_=tt_urv


In [7]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [8]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2025, 5, 13, 15, 19, 38, 842160)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [9]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [10]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [11]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [12]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")


### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [13]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [14]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, people generally liked John Wick. Many reviews highlight its stylish action, intense sequences, and entertainment value, with some rating it very highly (e.g., 9 or 10 out of 10). There are a few mixed or less favorable reviews, but overall, the sentiment indicates that people appreciated and enjoyed the film.'

In [15]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})

{'response': AIMessage(content="Based on the reviews provided, people generally liked John Wick. Many reviews give high ratings, praise the film's action sequences, style, and Keanu Reeves' performance, and describe it as fun, slick, and innovative. However, there are some lower-rated reviews and a few critics who are less enthusiastic. Overall, the majority of opinions seem to be positive, indicating that people generally appreciated and enjoyed the film.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 3595, 'total_tokens': 3678, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 3456}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45', 'id': 'chatcmpl-BXpGUUtx0KuWOJJhZKM8v8MBmaOA1', 'service_tier': 'default', 'finish_reason': 'stop', 'logpr

In [16]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. The URL to that review is: /review/rw4854296/?ref_=tt_urv'

In [17]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the John Wick film series, the story revolves around John Wick, a retired hitman who seeks vengeance after a series of personal tragedies. In the first movie, John comes out of retirement after gangsters kill his dog and steal his car, which were the last gifts from his deceased wife. His quest for retribution plunges him into the criminal underworld, where he is targeted by numerous assassins due to a bounty placed on his head. Throughout the series, John Wick navigates a dangerous world of criminal organizations, following a strict code of conduct, and facing the consequences of his violent past. The films are known for their stylish action sequences, deep world-building, and the recurring theme that "every action has consequences."'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [18]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [19]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [20]:
bm25_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, people have mixed opinions about John Wick. Some reviews are highly positive, praising the action, style, and entertainment value, suggesting that many people liked the movies, at least in the earlier installments. For example, one review gave a score of 10 and called it "something special," and another gave an 8, recommending it strongly.\n\nHowever, there are also negative reviews expressing dissatisfaction, especially with later installments like John Wick 3, which received a rating of 1 and was described as mindless and overly violent.\n\nOverall, while many people apparently enjoyed the John Wick movies, especially the first one, there are also notable criticisms, indicating that opinions vary.'

In [21]:
bm25_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'There are no reviews with a rating of 10 in the provided data.'

In [22]:
bm25_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In the John Wick film series, the story revolves around John Wick, a former assassin who is drawn back into the criminal underworld after a series of events. The first movie, John Wick 1, is highly praised for its beautifully choreographed action scenes and emotional depth. It follows Wick as he seeks vengeance after his dog is killed, which is a gift from his deceased wife. The subsequent films, John Wick 2 and John Wick 3, continue to explore his battles against assassins and crime organizations, featuring intense action and elaborate fight scenes. However, the later installment, John Wick 4, received some criticism for repetitive action sequences and implausible plot points, though it still maintains the core theme of Wick's relentless struggle for survival and justice."

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [23]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [24]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [25]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, people generally liked John Wick. The first two reviews are highly positive, praising the film as an exciting, stylish, and fun action movie with high ratings of 9 and 10. These reviews highlight Keanu Reeves' performance, the action sequences, and the overall entertainment value. However, a later review for John Wick 3 rated it lower at 5, indicating some disappointment, but overall, the sentiment from the positive reviews suggests that many viewers liked and appreciated the film."

In [26]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there are reviews with a rating of 10. Here are the URLs to those reviews:\n\n1. [Review titled "A Masterpiece & Brilliant Sequel"](https://yourwebsite.com/review/rw4854296/?ref_=tt_urv) - from john_wick_3.csv\n2. [Review simply titled "10"](https://yourwebsite.com/review/rw8944843/?ref_=tt_urv) - from john_wick_4.csv\n3. [Review titled "It\'s got its own action style!"](https://yourwebsite.com/review/rw4860412/?ref_=tt_urv) - from john_wick_3.csv\n\n(Note: The URLs are based on the provided review URLs; please replace "yourwebsite.com" with the actual domain if needed.)'

In [27]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In the John Wick films, John Wick (played by Keanu Reeves) is a retired hitman who is pulled back into a violent world of crime. The story begins when he resolves issues with the Russian mafia, but his peace is disrupted when a mobster named Santino D'Antonio visits him, demanding help with a favor represented by a marker. Wick refuses, leading Santino to blow up his house. Subsequently, Wick is asked to kill Santino's sister in Rome so he can sit on the criminal organization’s High Table. After completing this task, Santino puts a bounty on Wick’s head, making him the target of numerous killers. Wick then becomes motivated to seek revenge on Santino for the betrayal and the bounty placed on him. The series features intense action, stylish combat, and explores Wick’s struggle to navigate and survive in a dangerous criminal underworld."

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [28]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [29]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [30]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Yes, people generally liked John Wick. The reviews indicate high ratings and positive comments praising its action sequences, style, and overall entertainment value. Many reviewers gave it high scores (such as 9 or 10 out of 10) and described it as a fun, slick, and exciting action film that broke conventional molds.'

In [31]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a ratings of 10. The URL to that review is: /review/rw4854296/?ref_=tt_urv.'

In [32]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In the John Wick series, the story centers around John Wick, an ex-hitman who comes out of retirement to seek vengeance after personal tragedies. The first film starts with John mourning the death of his wife and the killing of his dog, which was a last gift from her. When a gangsters' arresting and theft of his car and dog occurs, Wick unleashes his lethal skills to exact revenge on those responsible, drawing the attention of the criminal underworld and bounty hunters who aim to eliminate him. Throughout the series, Wick is portrayed as a highly skilled and almost legendary assassin entangled in a world of organized crime, with each film exploring the consequences of his violent actions and his struggle for peace and survival amidst a complex network of criminal rules and rivalries."

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [33]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [34]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

/var/folders/8h/kl800c1j6hjc9xt9lm_1bhph0000gn/T/ipykernel_31838/3574430551.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [35]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [36]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [37]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [38]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the provided reviews, people generally liked the John Wick series. Many reviews are positive, praising the action, choreography, and overall entertainment value. However, there is at least one negative review that criticizes John Wick 4 specifically for its plot and action scenes. Overall, the sentiment leans toward liking the series, but opinions on the latest installment vary.'

In [39]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. The URL to that review is: /review/rw4854296/?ref_=tt_urv'

In [40]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In John Wick, the story centers around an ex-hitman named John Wick, played by Keanu Reeves, who comes out of retirement to seek vengeance after a series of personal losses and provocations. In the first film, someone steals his car and kills his dog, which prompts him to go on a violent rampage against those responsible, unleashing a deadly pursuit that draws in many assassins and reveals his lethal skills. The sequel, John Wick Chapter 2, continues his story as he is forced back into the criminal underworld to help a former associate. The plot involves travel to Italy, Canada, and Manhattan, and includes numerous violent encounters as Wick battles against other assassins and figures from his past, all while trying to escape the chaos he unleashes.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [41]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [42]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [43]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, people generally liked John Wick. Many reviews give high ratings (such as 8 or 9 out of 10) and describe the film as stylish, fun, and highly entertaining, especially appreciating its action sequences and Keanu Reeves' performance. However, there are some negative reviews with low ratings, criticizing the films for excess violence, implausible action, or lack of plot. Overall, the majority of the reviews suggest that viewers tend to enjoy John Wick, especially fans of action movies."

In [44]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

"Yes, there are reviews with a rating of 10. The URLs to those reviews are:\n\n1. [Review for John Wick 3](https://example.com/review/rw4854296/?ref_=tt_urv)\n2. [Review for John Wick 4](https://example.com/review/rw8947764/?ref_=tt_urv)\n\n(Note: The actual review URLs from the provided data are '/review/rw4854296/?ref_=tt_urv' and '/review/rw8947764/?ref_=tt_urv'. If you need the complete link, you may need to append the base URL of the site.)"

In [45]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In John Wick, a retired hitman named John Wick (played by Keanu Reeves) seeks revenge after a violent home invasion leaves him grieving the loss of his wife and the death of his beloved dog, which was his last remaining connection to her. The story begins with this personal tragedy, and Wick, initially attempting to rebuild his life, is drawn back into the violent underworld of assassins when a group of Russian mobsters, led by a punk who attacks him, steal his car and kill his dog. This act of violence triggers Wick's return to his former lethal skills as he unleashes a ruthless and meticulously orchestrated campaign of revenge against those who wronged him. Throughout the series, Wick is targeted by numerous bounty hunters and assassins, leading to intense action sequences and a complex web of criminal alliances and rules. The films explore themes of vengeance, consequence, and the dangerous world of professional killers."

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

> NOTE: You do not need to run this cell if you're running this locally

In [46]:
#!pip install -qU langchain_experimental

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [47]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [48]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [49]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [50]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [51]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [52]:
semantic_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, people generally liked John Wick. The reviews are mostly positive, highlighting the film's stylish action, choreography, and entertainment value. However, there are some mixed opinions as well. Overall, the majority of comments suggest that John Wick was well-received and appreciated by audiences."

In [53]:
semantic_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. The URL to that review is: /review/rw4854296/?ref_=tt_urv'

In [54]:
semantic_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In the John Wick movies, the story revolves around a retired and highly skilled assassin named John Wick. After the death of his wife, he is found to be seeking peace in his life. However, his life takes a violent turn when a young Russian punk, whom Wick declines to sell his classic car to, and his gang surprise him at his home. They beat him up, kill his beloved dog, and steal his car. \n\nThis brutal attack awakens Wick'skiller instincts, revealing his background as a super-assassin. Enraged and seeking revenge, Wick embarks on a relentless quest against the gangsters who wronged him, including their mobster father. The films showcase his swift, stylish, and often brutal action as he fights to reclaim his peace and deal with the consequences of his past. The story emphasizes themes of vengeance, consequence, and the dangerous world of hired killers."

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [55]:
### YOUR CODE HERE

In [118]:
# Imports standard
import copy
import os
import time
import uuid
import numpy as np
import pandas as pd
import tiktoken

# Import OpenAI
from openai import OpenAI

# Imports LangChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate


from langchain.schema.runnable import RunnableConfig
from langchain_core.tracers.context import tracing_v2_enabled
from langsmith import Client
# Imports LangSmith
from langsmith import Client, traceable, RunTree
from langsmith.run_helpers import get_current_run_tree
from langsmith.wrappers import wrap_openai

# Imports RAGAS
from ragas import EvaluationDataset, evaluate, RunConfig
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset import TestsetGenerator
from ragas.metrics import (
    LLMContextRecall,
    Faithfulness, 
    FactualCorrectness,
    ResponseRelevancy,
    ContextEntityRecall
)

In [ ]:
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
testset_dataset = generator.generate_with_langchain_docs(documents, testset_size=10)

In [119]:
testset_df = testset_dataset.to_pandas()
testset_df

,user_input,reference_contexts,reference,synthesizer_name
0,Why John Wick movie so good even tho story sim...,[: 0\nReview: The best way I can describe John...,John Wick is good because it has a simple stor...,single_hop_specifc_query_synthesizer
1,"Could you provide an overview of the ""John Wic...",[: 2\nReview: With the fourth installment scor...,"The ""John Wick"" film series has gained signifi...",single_hop_specifc_query_synthesizer
2,Who is Chad Stahelski and why he important for...,[: 3\nReview: John wick has a very simple reve...,Chad Stahelski is the director of John Wick an...,single_hop_specifc_query_synthesizer
3,Can you explane the plot and action style of t...,[: 4\nReview: Though he no longer has a taste ...,John Wick follows a retired assassin who has l...,single_hop_specifc_query_synthesizer
4,How does the escalation of stakes in the John ...,[<1-hop>\n\n: 0\nReview: It is 5 years since t...,The escalation of stakes in the John Wick seri...,multi_hop_abstract_query_synthesizer
5,Is Keanu Reeves acting in John Wick: Chapter 4...,[<1-hop>\n\n: 23\nReview: Rating 10/10\nI was ...,Keanu Reeves' acting in John Wick: Chapter 4 r...,multi_hop_abstract_query_synthesizer
6,How does the depiction of katana swords versus...,[<1-hop>\n\n: 13\nReview: From the very beginn...,"In the fourth chapter, the depiction of katana...",multi_hop_abstract_query_synthesizer
7,How do the film reviews reflect the portrayal ...,[<1-hop>\n\n: 20\nReview: John Wick is somethi...,The first review praises John Wick as a specia...,multi_hop_abstract_query_synthesizer
8,How does Ian McShane's role in John Wick contr...,"[<1-hop>\n\n: 9\nReview: At first glance, John...",Ian McShane's role in John Wick is pivotal to ...,multi_hop_specific_query_synthesizer
9,Why do some action movie fans like Taken but t...,[<1-hop>\n\n: 14\nReview: I absolutely love ac...,"Some action movie fans love Taken, but they fi...",multi_hop_specific_query_synthesizer


In [120]:
retrievers = [
    {"name": "Naive Retriever", "retriever": naive_retriever},
    {"name": "BM25 Retriever", "retriever": bm25_retriever},
    {"name": "Contextual Compression", "retriever": compression_retriever},
    {"name": "Multi-Query Retriever", "retriever": multi_query_retriever},
    {"name": "Parent Document Retriever", "retriever": parent_document_retriever},
    {"name": "Ensemble Retriever", "retriever": ensemble_retriever},
    {"name": "Semantic Chunking Retriever", "retriever": semantic_retriever}
]

eval_llm = ChatOpenAI(model="gpt-4.1-mini")


In [135]:
# Vérification de la version de LangSmith
import langsmith
print(f"LangSmith version: {langsmith.__version__}")

# Vérification des variables d'environnement
print(f"LANGCHAIN_TRACING_V2: {os.environ.get('LANGCHAIN_TRACING_V2', 'NOT SET')}")
print(f"LANGSMITH_API_KEY: {'SET' if os.environ.get('LANGSMITH_API_KEY') else 'NOT SET'}")
print(f"LANGSMITH_TRACING: {os.environ.get('LANGSMITH_TRACING', 'NOT SET')}")

# S'assurer que le traçage est activé
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_TRACING"] = "true"

client = Client()
try:
    # Vérifier que la connexion à l'API fonctionne
    project_count = 0
    for project in client.list_projects():
        project_count += 1
        if project_count <= 3:  # Limitons à 3 pour éviter de tout afficher
            print(f"Project found: {project.name}")
    
    print(f"LangSmith connection successful! Found {project_count} projects.")
    
    # Afficher la configuration de l'API
    print(f"API Base URL: {client.api_url}")
    if client.api_key:
        masked_key = client.api_key[:4] + "..." + client.api_key[-4:]
        print(f"API Key: {masked_key}")
except Exception as e:
    print(f"Error connecting to LangSmith: {e}")

# Configuration de l'encodeur pour compter les tokens
encoding = tiktoken.get_encoding("cl100k_base")

# Définition des coûts
GENERATION_INPUT_COST = 0.0000001  # $0.10 par million de tokens
GENERATION_OUTPUT_COST = 0.0000004  # $0.40 par million de tokens

LangSmith version: 0.3.42
LANGCHAIN_TRACING_V2: true
LANGSMITH_API_KEY: SET
LANGSMITH_TRACING: true
Project found: retriever-eval-bm25-retriever-4a6e189d
Project found: retriever-eval-naive-retriever-96786722
Project found: retriever-eval-naive-retriever-efb381ba
LangSmith connection successful! Found 69 projects.
API Base URL: https://api.smith.langchain.com
API Key: lsv2...76ee


In [112]:
# api_key = getpass.getpass("Entrez votre clé API LangSmith: ")
# os.environ["LANGSMITH_API_KEY"] = api_key
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [ ]:

#US Personnal
#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



In [133]:
import os
import time
import uuid
import numpy as np
import pandas as pd
import tiktoken
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks import get_openai_callback
from langsmith import Client, traceable
from langsmith.wrappers import wrap_openai

# Configuration de l'encodeur pour compter les tokens
encoding = tiktoken.get_encoding("cl100k_base")

# Définition des coûts
GENERATION_INPUT_COST = 0.0000001  # $0.10 par million de tokens
GENERATION_OUTPUT_COST = 0.0000004  # $0.40 par million de tokens

def evaluate_retrievers_complete(retrievers, testset_dataset, llm_model="gpt-4.1-nano"):
    """
    Évaluation unifiée des retrievers avec traçage LangSmith, 
    incluant l'évaluation RAGAS pour chaque retriever dans le même workflow.
    """
    # Client LangSmith
    client = Client(api_key=os.environ.get("LANGSMITH_API_KEY"))os.environ.get("LANGSMITH_API_KEY"))
    results = []
    
    # Template RAG
    prompt_template = """
    You are a helpful assistant. Use the context provided below to answer the question.
    If you don't know the answer, just say you don't know.
    
    Context:
    {context}
    
    Question:
    {question}
    """
    
    # Évaluer chaque retriever
    for retriever_info in retrievers:
        name = retriever_info["name"]
        retriever = retriever_info["retriever"]
        
        print(f"\n=== Evaluating: {name} ===")
        
        # Créer un projet unique pour ce retriever
        project_name = f"retriever-eval-{name.lower().replace(' ', '-')}-{uuid.uuid4().hex[:8]}"
        try:
            client.create_project(project_name=project_name)
            print(f"Project: {project_name}")
        except Exception as e:
            print(f"Could not create project: {e}")
            project_name = "default"
        
        # Client OpenAI wrappé
        openai_client = wrap_openai(OpenAI())
        
        # Évaluation complète du retriever
        @traceable(
            project_name=project_name,
            run_type="chain",
            name=f"{name} Complete Evaluation"
        )
        def evaluate_retriever_with_ragas():
            metrics = {
                "retrieval_times": [],
                "llm_times": [],
                "total_times": [],
                "document_counts": [],
                "prompt_tokens": [],
                "completion_tokens": [],
                "costs": []
            }
            
            # Copie du testset
            testset_copy = copy.deepcopy(testset_dataset)
            
            # Traiter chaque question
            for i, test_row in enumerate(testset_copy):
                question = test_row.eval_sample.user_input
                print(f"Processing question {i+1}/{len(testset_copy)}: {question[:50]}...")
                
                # Fonction interne pour traiter chaque question
                @traceable(
                    run_type="chain",
                    name=f"Question {i+1}",
                    metadata={"question_id": i}
                )
                def process_question(query):
                    # 1. Récupération
                    retrieval_start = time.time()
                    docs = retriever.invoke(query)
                    retrieval_time = time.time() - retrieval_start
                    
                    # Métriques de récupération
                    doc_count = len(docs)
                    metrics["retrieval_times"].append(retrieval_time)
                    metrics["document_counts"].append(doc_count)
                    
                    # 2. Préparation du contexte
                    context_text = "\n\n".join([doc.page_content for doc in docs])
                    formatted_prompt = prompt_template.format(context=context_text, question=query)
                    
                    # Calcul des tokens
                    prompt_tokens = len(encoding.encode(formatted_prompt))
                    metrics["prompt_tokens"].append(prompt_tokens)
                    
                    # 3. Appel LLM
                    llm_start = time.time()
                    llm_response = openai_client.chat.completions.create(
                        model=llm_model,
                        messages=[
                            {"role": "system", "content": formatted_prompt}
                        ]
                    )
                    llm_time = time.time() - llm_start
                    metrics["llm_times"].append(llm_time)
                    
                    # Réponse
                    answer = llm_response.choices[0].message.content
                    
                    # Métriques de tokens et coûts
                    if hasattr(llm_response, "usage") and llm_response.usage:
                        completion_tokens = llm_response.usage.completion_tokens
                        metrics["completion_tokens"].append(completion_tokens)
                        
                        cost = (
                            (llm_response.usage.prompt_tokens * GENERATION_INPUT_COST) + 
                            (completion_tokens * GENERATION_OUTPUT_COST)
                        )
                        metrics["costs"].append(cost)
                    
                    # Temps total
                    total_time = retrieval_time + llm_time
                    metrics["total_times"].append(total_time)
                    
                    return {
                        "response": answer,
                        "contexts": [doc.page_content for doc in docs],
                        "retrieval_time": retrieval_time,
                        "llm_time": llm_time,
                        "total_time": total_time,
                        "document_count": doc_count
                    }
                
                # Traiter la question
                try:
                    result = process_question(question)
                    
                    # Mettre à jour le testset
                    test_row.eval_sample.response = result["response"]
                    test_row.eval_sample.retrieved_contexts = result["contexts"]
                    
                    # Afficher les métriques
                    print(f"  Time: {result['total_time']:.2f}s (Retrieval: {result['retrieval_time']:.2f}s, LLM: {result['llm_time']:.2f}s)")
                    print(f"  Documents: {result['document_count']}, Cost: ${metrics['costs'][-1] if metrics['costs'] else 0:.6f}")
                    
                except Exception as e:
                    print(f"Error processing question {i}: {e}")
                    import traceback
                    traceback.print_exc()
                
                # Pause pour éviter de surcharger l'API
                time.sleep(0.5)
            
            # Calculer les moyennes
            avg_metrics = {
                "avg_retrieval_time": np.mean(metrics["retrieval_times"]) if metrics["retrieval_times"] else 0,
                "avg_llm_time": np.mean(metrics["llm_times"]) if metrics["llm_times"] else 0,
                "avg_total_time": np.mean(metrics["total_times"]) if metrics["total_times"] else 0,
                "avg_docs": np.mean(metrics["document_counts"]) if metrics["document_counts"] else 0,
                "avg_prompt_tokens": np.mean(metrics["prompt_tokens"]) if metrics["prompt_tokens"] else 0,
                "avg_completion_tokens": np.mean(metrics["completion_tokens"]) if metrics["completion_tokens"] else 0,
                "avg_cost": np.mean(metrics["costs"]) if metrics["costs"] else 0
            }
            
            print("\n=== Running RAGAS evaluation ===")
            
            # Exécuter RAGAS dans le même flux de traçage
            @traceable(
                run_type="evaluation",
                name="RAGAS Evaluation"
            )
            def run_ragas_evaluation():
                try:
                    # Import RAGAS
                    from ragas.llms import LangchainLLMWrapper
                    from ragas import EvaluationDataset
                    from ragas.metrics import (
                        LLMContextRecall,
                        Faithfulness,
                        FactualCorrectness,
                        ResponseRelevancy,
                        ContextEntityRecall
                    )
                    from ragas import evaluate, RunConfig
                    
                    # Configuration RAGAS
                    evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model=llm_model))
                    run_config = RunConfig(timeout=600, max_workers=2)
                    
                    # Métriques RAGAS
                    metrics = [
                        LLMContextRecall(),
                        Faithfulness(),
                        FactualCorrectness(),
                        ResponseRelevancy(),
                        ContextEntityRecall()
                    ]
                    
                    # Préparation du dataset
                    ragas_df = pd.DataFrame()
                    df = pd.DataFrame([
                        {
                            "question": row.eval_sample.user_input,
                            "answer": row.eval_sample.response,
                            "contexts": row.eval_sample.retrieved_contexts
                        }
                        for row in testset_copy
                    ])
                    
                    ragas_df['question'] = df['question']
                    ragas_df['answer'] = df['answer']
                    ragas_df['contexts'] = df['contexts'].apply(
                        lambda x: x if isinstance(x, list) else [x]
                    )
                    ragas_df['ground_truths'] = [[] for _ in range(len(df))]
                    
                    # Conversion en dataset RAGAS
                    dataset = EvaluationDataset.from_pandas(ragas_df)
                    
                    # Exécution de l'évaluation
                    result = evaluate(
                        dataset=dataset,
                        metrics=metrics,
                        llm=evaluator_llm,
                        run_config=run_config
                    )
                    
                    # Calcul des scores moyens
                    ragas_scores = {}
                    ragas_scores['llm_context_recall'] = np.mean(result.get('llm_context_recall', [0]))
                    ragas_scores['faithfulness'] = np.mean(result.get('faithfulness', [0]))
                    ragas_scores['factual_correctness'] = np.mean(result.get('factual_correctness(mode=f1)', [0]))
                    ragas_scores['response_relevancy'] = np.mean(result.get('answer_relevancy', [0]))
                    ragas_scores['context_entity_recall'] = np.mean(result.get('context_entity_recall', [0]))
                    
                    # Score global
                    ragas_scores['quality_score'] = np.mean([
                        ragas_scores['llm_context_recall'],
                        ragas_scores['faithfulness'],
                        ragas_scores['factual_correctness'],
                        ragas_scores['response_relevancy'],
                        ragas_scores['context_entity_recall']
                    ])
                    
                    # Afficher les scores
                    print(f"RAGAS scores for {name}:")
                    for key, value in ragas_scores.items():
                        print(f"- {key}: {value:.3f}")
                    
                    return ragas_scores
                    
                except Exception as e:
                    print(f"Error in RAGAS evaluation: {e}")
                    import traceback
                    traceback.print_exc()
                    return {"quality_score": 0.5}
            
            # Exécuter RAGAS
            ragas_scores = run_ragas_evaluation()
            
            # Combiner les métriques et retourner
            return {
                "metrics": avg_metrics,
                "ragas_scores": ragas_scores,
                "processed_testset": testset_copy
            }
        
        # Exécuter l'évaluation complète
        evaluation_result = evaluate_retriever_with_ragas()
        
        # Extraire les résultats
        avg_metrics = evaluation_result["metrics"]
        ragas_scores = evaluation_result["ragas_scores"]
        processed_testset = evaluation_result["processed_testset"]
        
        # Stocker les résultats pour ce retriever
        retriever_result = {
            "Retriever": name,
            "Avg Time (s)": avg_metrics["avg_total_time"],
            "Retrieval Time (s)": avg_metrics["avg_retrieval_time"],
            "LLM Time (s)": avg_metrics["avg_llm_time"],
            "Avg Docs": avg_metrics["avg_docs"],
            "Avg Prompt Tokens": avg_metrics["avg_prompt_tokens"],
            "Avg Completion Tokens": avg_metrics["avg_completion_tokens"],
            "Avg Cost ($)": avg_metrics["avg_cost"],
            "Quality Score": ragas_scores.get("quality_score", 0.5),
            "LLM Context Recall": ragas_scores.get("llm_context_recall", 0),
            "Faithfulness": ragas_scores.get("faithfulness", 0),
            "Factual Correctness": ragas_scores.get("factual_correctness", 0),
            "Response Relevancy": ragas_scores.get("response_relevancy", 0),
            "Context Entity Recall": ragas_scores.get("context_entity_recall", 0),
            "Project": project_name
        }
        
        # Calculer les métriques d'efficacité
        retriever_result["Efficiency Score"] = retriever_result["Quality Score"] / max(retriever_result["Avg Cost ($)"], 0.0001)
        retriever_result["Time-Efficiency"] = retriever_result["Quality Score"] / (max(retriever_result["Avg Cost ($)"], 0.0001) * retriever_result["Avg Time (s)"])
        
        # Ajouter aux résultats
        results.append(retriever_result)
        
        # Afficher les résultats
        print(f"\n=== Results for {name} ===")
        print(f"Average time: {avg_metrics['avg_total_time']:.2f}s (Retrieval: {avg_metrics['avg_retrieval_time']:.2f}s, LLM: {avg_metrics['avg_llm_time']:.2f}s)")
        print(f"Average documents: {avg_metrics['avg_docs']:.1f}")
        print(f"Average tokens: {avg_metrics['avg_prompt_tokens']:.1f} in, {avg_metrics['avg_completion_tokens']:.1f} out")
        print(f"Average cost: ${avg_metrics['avg_cost']:.6f}/query")
        print(f"Quality Score: {ragas_scores.get('quality_score', 0.5):.3f}")
    
    # Convertir en DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

In [134]:
 #Exécuter l'évaluation complète en une seule fois
results_df = evaluate_retrievers_complete(
    retrievers, 
    testset_dataset,
    llm_model="gpt-4.1-nano"
)

# Afficher les résultats finaux
print("\n=== Final Results ===")
print(results_df[["Retriever", "Avg Time (s)", "Avg Cost ($)", "Quality Score", "Efficiency Score"]])


=== Evaluating: Naive Retriever ===
Project: retriever-eval-naive-retriever-96786722
Processing question 1/12: Why John Wick movie so good even tho story simple ...
  Time: 3.57s (Retrieval: 0.63s, LLM: 2.94s)
  Documents: 10, Cost: $0.000276
Processing question 2/12: Could you provide an overview of the "John Wick" f...
  Time: 5.51s (Retrieval: 0.73s, LLM: 4.78s)
  Documents: 10, Cost: $0.000303
Processing question 3/12: Who is Chad Stahelski and why he important for Joh...
  Time: 2.34s (Retrieval: 0.28s, LLM: 2.06s)
  Documents: 10, Cost: $0.000263
Processing question 4/12: Can you explane the plot and action style of the m...
  Time: 4.97s (Retrieval: 0.36s, LLM: 4.61s)
  Documents: 10, Cost: $0.000383
Processing question 5/12: How does the escalation of stakes in the John Wick...
  Time: 2.72s (Retrieval: 0.30s, LLM: 2.42s)
  Documents: 10, Cost: $0.000231
Processing question 6/12: Is Keanu Reeves acting in John Wick: Chapter 4 pra...
  Time: 1.50s (Retrieval: 0.21s, LLM: 1.29s)

/Users/jthomazo/Archives/01_Projets/02_AIM/AIE6/13_Advanced_Retrieval/.venv/lib/python3.13/site-packages/langsmith/run_helpers.py:452: UserWarning: Unrecognized run_type: evaluation. Must be one of: {'parser', 'retriever', 'prompt', 'tool', 'llm', 'chain', 'embedding'}. Did you mean @traceable(name='evaluation')?
  warnings.warn(


Error in RAGAS evaluation: The metric [context_recall] that is used requires the following additional columns ['user_input', 'retrieved_contexts', 'reference'] to be present in the dataset.

=== Results for Naive Retriever ===
Average time: 3.78s (Retrieval: 0.46s, LLM: 3.32s)
Average documents: 10.0
Average tokens: 2028.4 in, 190.5 out
Average cost: $0.000276/query
Quality Score: 0.500

=== Evaluating: BM25 Retriever ===


Traceback (most recent call last):
  File "/var/folders/8h/kl800c1j6hjc9xt9lm_1bhph0000gn/T/ipykernel_31838/911973947.py", line 236, in run_ragas_evaluation
    result = evaluate(
        dataset=dataset,
    ...<2 lines>...
        run_config=run_config
    )
  File "/Users/jthomazo/Archives/01_Projets/02_AIM/AIE6/13_Advanced_Retrieval/.venv/lib/python3.13/site-packages/ragas/_analytics.py", line 227, in wrapper
    result = func(*args, **kwargs)
  File "/Users/jthomazo/Archives/01_Projets/02_AIM/AIE6/13_Advanced_Retrieval/.venv/lib/python3.13/site-packages/ragas/evaluation.py", line 176, in evaluate
    validate_required_columns(dataset, metrics)
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "/Users/jthomazo/Archives/01_Projets/02_AIM/AIE6/13_Advanced_Retrieval/.venv/lib/python3.13/site-packages/ragas/validation.py", line 63, in validate_required_columns
    raise ValueError(
    ...<3 lines>...
    )
ValueError: The metric [context_recall] that is used requires the followin

Project: retriever-eval-bm25-retriever-4a6e189d
Processing question 1/12: Why John Wick movie so good even tho story simple ...
  Time: 4.59s (Retrieval: 0.00s, LLM: 4.59s)
  Documents: 4, Cost: $0.000238
Processing question 2/12: Could you provide an overview of the "John Wick" f...


KeyboardInterrupt: 